<a href="https://colab.research.google.com/github/PyDataOsaka/handson_pytorch/blob/master/resnet18_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pytorch hands-on (Resnet18)

Code of Resnet18 was adapted from [here](https://github.com/pytorch/xla/blob/master/contrib/colab/resnet18-training.ipynb)

In [0]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [0]:
os.environ["XLA_USE_BF16"] = "1"

In [0]:
!rm -r ./log

In [0]:
%tensorflow_version 2.x
%load_ext tensorboard

In [0]:
%tensorboard --logdir ./log

## Installing Pytorch/XLA

In [0]:
VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

In [0]:
from time import time
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision
import torchvision.transforms as transforms
import torch_xla
import torch_xla.core.xla_model as xm

## Load image data

In [0]:
def get_data(batch_size: int=64):
  transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=2)

  testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                         download=True, transform=transform)
  testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                           shuffle=False, num_workers=2)

  classes = ('plane', 'car', 'bird', 'cat',
             'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
  
  return trainloader, testloader, classes

## Resnet18

In [0]:
class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_planes, planes, stride=1):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(
        in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(
        planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != self.expansion * planes:
      self.shortcut = nn.Sequential(
        nn.Conv2d(
          in_planes,
          self.expansion * planes,
          kernel_size=1,
          stride=stride,
          bias=False), nn.BatchNorm2d(self.expansion * planes))

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out


class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    self.in_planes = 64

    self.conv1 = nn.Conv2d(
        3, 64, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(64)
    self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
    self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
    self.linear = nn.Linear(512 * block.expansion, num_classes)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1] * (num_blocks - 1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion
    return nn.Sequential(*layers)

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = self.layer4(out)
    out = F.avg_pool2d(out, 4)
    out = torch.flatten(out, 1)
    out = self.linear(out)
    return F.log_softmax(out, dim=1)

def ResNet18():
  return ResNet(BasicBlock, [2, 2, 2, 2])

## Define functions

### Training

In [0]:
def train(model: nn.Module, trainloader, log_dir: str, device: str="cpu",
          n_epochs=4):
  model.to(device)

  loss = nn.CrossEntropyLoss()
  opt = optim.SGD(model.parameters(), lr=0.02, momentum=0.9, weight_decay=5e-4)

  writer = SummaryWriter(log_dir)
  running_loss = 0.0
  prev_time = time()
  n_minibatches = 0
  terminate_for_cpu = False

  for epoch in range(n_epochs):
    # When using CPU the loop will be terminated after a few steps
    if terminate_for_cpu:
      break

    for i, data in enumerate(trainloader, 0):
      # get the inputs; data is a list of [inputs, labels]
      inputs = data[0].to(device)
      labels = data[1].to(device)

      # zero the parameter gradients
      opt.zero_grad()

      # forward + backward + optimize
      outputs = model(inputs)
      loss_value = loss(outputs, labels)
      loss_value.backward()
      # opt.step() # For CPU/GPU
      xm.optimizer_step(opt, barrier=True)  # Note: Cloud TPU-specific code!

      writer.add_scalar("loss_value", loss_value, n_minibatches)
      n_minibatches += 1

      # print statistics
      running_loss += loss_value.item()
      if device == "cpu":
        if i == 4:
          dt = time() - prev_time
          print("Elapsed time: {:.1f} [sec] for 5 training steps".format(dt))
          print("({:.1f} [sec] for 100 updates)".format(20 * dt))
          terminate_for_cpu = True
          break
      else:
        if i % 100 == 99:    # print every 100 mini-batches
          print('[{}, {:5d}] loss: {:.3f}, elapsed time: {:.1f} [sec]'.format(
                epoch + 1, i + 1, running_loss / 2000, time() - prev_time))
          running_loss = 0.0
          prev_time = time()

  writer.close()

### Prediction

In [0]:
def evaluate(model: nn.Module, testloader, device: str="cpu"):
  correct = 0
  total = 0

  with torch.no_grad():
    for data in testloader:
      inputs = data[0].to(device)
      labels = data[1].to(device)
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += predicted.eq(labels.view_as(predicted)).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
          100 * correct / total))

## Training and evaluation of the model on TPU

In [0]:
trainloader, testloader, classes = get_data()
model = ResNet18()

dev = xm.xla_device()
model.train()
train(model, trainloader, "./log/3", dev)
model.eval()
evaluate(model, testloader, dev)

In [0]:
!nvidia-smi